## Initial Setup

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os
import math
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

import warnings
warnings.filterwarnings("ignore")

## Training Parameters

In [2]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    # GPU
    BATCH_SIZE = 256  # Number of examples used in each iteration
    EPOCHS = 10  # Number of passes through entire dataset
    MAX_LEN = 170  # Max length of review (in words)
    VOCAB_SIZE = 1000  # Size of vocabulary dictionary
    EMBEDDING = 8  # Dimension of word embedding vector
    
# Hyperparams for CPU training
else:
    # CPU
    BATCH_SIZE = 128
    EPOCHS = 10
    MAX_LEN = 170
    VOCAB_SIZE = 1000
    EMBEDDING = 8

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


## Data

In [7]:
data=pd.read_excel(r"C:\Users\User\Desktop\‎ ‎ ‎ ‎ ‎ ‎ ‎           ‎\Projects\Dr Andrew (Ongoing)\Shedrach\Xantham_gum_data.xlsx")
data


,X1,X2,X3,Observed
0,1,2,7.5,7.22
1,3,2,7.5,7.52
2,1,6,7.5,4.58
3,3,6,7.5,6.45
4,1,4,0.0,4.47
5,3,4,0.0,5.67
6,1,4,15.0,6.46
7,3,4,15.0,7.76
8,2,2,0.0,6.65
9,2,6,0.0,4.52


## Data Preprocessing

In [8]:
target="Observed"
X=data.drop(columns=[target])
y=data[target]
X1=X.values
y1=y.values.reshape(-1,1)

In [11]:
y.values.reshape(-1,1)

array([[7.22],
       [7.52],
       [4.58],
       [6.45],
       [4.47],
       [5.67],
       [6.46],
       [7.76],
       [6.65],
       [4.52],
       [8.31],
       [6.93],
       [5.81],
       [6.21],
       [6.28],
       [6.22],
       [5.86]])

In [12]:
def get_preds(y_test, y_preds):
    y_test=pd.DataFrame(y_test)
    y_test.rename(columns={0:'Actual'}, inplace=True)
    y_preds=pd.DataFrame(y_preds)
    y_preds.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([y_test, y_preds], axis=1)
    return predictions

In [13]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score,make_scorer
X_train, X_test, y_train, y_test= train_test_split(X1, y1, test_size=0.2, random_state=66)

In [14]:
sc_x=StandardScaler()
X_train=sc_x.fit_transform(X_train)
X_test=sc_x.transform(X_test)
X1=sc_x.transform(X1)
sc_y=StandardScaler()
y_train=sc_y.fit_transform(y_train)
y_test=sc_y.transform(y_test)
y1=sc_y.transform(y1)

In [15]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [16]:
NN_model = Sequential()

In [17]:
# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(16, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               512       
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [18]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [19]:
import time
start_time=time.time()
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8341 - mean_absolute_error: 0.8341
Epoch 1: val_loss improved from inf to 0.18403, saving model to Weights-001--0.18403.hdf5
1/1 [==============================] - 3s 3s/step - loss: 0.8341 - mean_absolute_error: 0.8341 - val_loss: 0.1840 - val_mean_absolute_error: 0.1840
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8337 - mean_absolute_error: 0.8337
Epoch 2: val_loss did not improve from 0.18403
1/1 [==============================] - 0s 93ms/step - loss: 0.8337 - mean_absolute_error: 0.8337 - val_loss: 0.1851 - val_mean_absolute_error: 0.1851
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8333 - mean_absolute_error: 0.8333
Epoch 3: val_loss did not improve from 0.18403
1/1 [==============================] - 0s 76ms/step - loss: 0.8333 - mean_absolute_error: 0.8333 - val_loss: 0.1864 - val_mean_absolute_error: 0.1864
Epoch 4/500
1/1 [==============================] - ETA: 

Epoch 28/500
1/1 [==============================] - ETA: 0s - loss: 0.7991 - mean_absolute_error: 0.7991
Epoch 28: val_loss did not improve from 0.18403
1/1 [==============================] - 0s 47ms/step - loss: 0.7991 - mean_absolute_error: 0.7991 - val_loss: 0.2170 - val_mean_absolute_error: 0.2170
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.7960 - mean_absolute_error: 0.7960
Epoch 29: val_loss did not improve from 0.18403
1/1 [==============================] - 0s 48ms/step - loss: 0.7960 - mean_absolute_error: 0.7960 - val_loss: 0.2171 - val_mean_absolute_error: 0.2171
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.7925 - mean_absolute_error: 0.7925
Epoch 30: val_loss did not improve from 0.18403
1/1 [==============================] - 0s 48ms/step - loss: 0.7925 - mean_absolute_error: 0.7925 - val_loss: 0.2174 - val_mean_absolute_error: 0.2174
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.7884 - mean_absolute_

1/1 [==============================] - 0s 180ms/step - loss: 0.4323 - mean_absolute_error: 0.4323 - val_loss: 0.1388 - val_mean_absolute_error: 0.1388
Epoch 55/500
1/1 [==============================] - ETA: 0s - loss: 0.4092 - mean_absolute_error: 0.4092
Epoch 55: val_loss improved from 0.13876 to 0.12115, saving model to Weights-055--0.12115.hdf5
1/1 [==============================] - 0s 152ms/step - loss: 0.4092 - mean_absolute_error: 0.4092 - val_loss: 0.1212 - val_mean_absolute_error: 0.1212
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.4018 - mean_absolute_error: 0.4018
Epoch 56: val_loss improved from 0.12115 to 0.09622, saving model to Weights-056--0.09622.hdf5
1/1 [==============================] - 0s 169ms/step - loss: 0.4018 - mean_absolute_error: 0.4018 - val_loss: 0.0962 - val_mean_absolute_error: 0.0962
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.3866 - mean_absolute_error: 0.3866
Epoch 57: val_loss improved from 0.09622 t

Epoch 81/500
1/1 [==============================] - ETA: 0s - loss: 0.0957 - mean_absolute_error: 0.0957
Epoch 81: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 60ms/step - loss: 0.0957 - mean_absolute_error: 0.0957 - val_loss: 0.0287 - val_mean_absolute_error: 0.0287
Epoch 82/500
1/1 [==============================] - ETA: 0s - loss: 0.0833 - mean_absolute_error: 0.0833
Epoch 82: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 61ms/step - loss: 0.0833 - mean_absolute_error: 0.0833 - val_loss: 0.0066 - val_mean_absolute_error: 0.0066
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.0748 - mean_absolute_error: 0.0748
Epoch 83: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0748 - mean_absolute_error: 0.0748 - val_loss: 0.0103 - val_mean_absolute_error: 0.0103
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.0565 - mean_absolute_

Epoch 108/500
1/1 [==============================] - ETA: 0s - loss: 0.0216 - mean_absolute_error: 0.0216
Epoch 108: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0216 - mean_absolute_error: 0.0216 - val_loss: 0.0354 - val_mean_absolute_error: 0.0354
Epoch 109/500
1/1 [==============================] - ETA: 0s - loss: 0.0283 - mean_absolute_error: 0.0283
Epoch 109: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.0283 - mean_absolute_error: 0.0283 - val_loss: 0.0607 - val_mean_absolute_error: 0.0607
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0192 - mean_absolute_error: 0.0192
Epoch 110: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.0192 - mean_absolute_error: 0.0192 - val_loss: 0.0639 - val_mean_absolute_error: 0.0639
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0182 - mean_ab

...

Epoch 378/500
1/1 [==============================] - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.0057
Epoch 378: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 37ms/step - loss: 0.0057 - mean_absolute_error: 0.0057 - val_loss: 0.0629 - val_mean_absolute_error: 0.0629
Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0074 - mean_absolute_error: 0.0074
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 42ms/step - loss: 0.0074 - mean_absolute_error: 0.0074 - val_loss: 0.0606 - val_mean_absolute_error: 0.0606
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0071 - mean_absolute_error: 0.0071
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 37ms/step - loss: 0.0071 - mean_absolute_error: 0.0071 - val_loss: 0.0623 - val_mean_absolute_error: 0.0623
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0096 - mean_ab

Epoch 405/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_absolute_error: 0.0087
Epoch 405: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 40ms/step - loss: 0.0087 - mean_absolute_error: 0.0087 - val_loss: 0.0633 - val_mean_absolute_error: 0.0633
Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.0078
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 38ms/step - loss: 0.0078 - mean_absolute_error: 0.0078 - val_loss: 0.0678 - val_mean_absolute_error: 0.0678
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0056 - mean_absolute_error: 0.0056
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 37ms/step - loss: 0.0056 - mean_absolute_error: 0.0056 - val_loss: 0.0650 - val_mean_absolute_error: 0.0650
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0040 - mean_ab

Epoch 432/500
1/1 [==============================] - ETA: 0s - loss: 0.0072 - mean_absolute_error: 0.0072
Epoch 432: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 46ms/step - loss: 0.0072 - mean_absolute_error: 0.0072 - val_loss: 0.0693 - val_mean_absolute_error: 0.0693
Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0099 - mean_absolute_error: 0.0099
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0099 - mean_absolute_error: 0.0099 - val_loss: 0.0629 - val_mean_absolute_error: 0.0629
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0072 - mean_absolute_error: 0.0072
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 42ms/step - loss: 0.0072 - mean_absolute_error: 0.0072 - val_loss: 0.0617 - val_mean_absolute_error: 0.0617
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0056 - mean_ab

Epoch 459/500
1/1 [==============================] - ETA: 0s - loss: 0.0124 - mean_absolute_error: 0.0124
Epoch 459: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 37ms/step - loss: 0.0124 - mean_absolute_error: 0.0124 - val_loss: 0.0678 - val_mean_absolute_error: 0.0678
Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0101 - mean_absolute_error: 0.0101
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 40ms/step - loss: 0.0101 - mean_absolute_error: 0.0101 - val_loss: 0.0722 - val_mean_absolute_error: 0.0722
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0127 - mean_absolute_error: 0.0127
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 34ms/step - loss: 0.0127 - mean_absolute_error: 0.0127 - val_loss: 0.0674 - val_mean_absolute_error: 0.0674
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0080 - mean_ab

Epoch 486/500
1/1 [==============================] - ETA: 0s - loss: 0.0154 - mean_absolute_error: 0.0154
Epoch 486: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0154 - mean_absolute_error: 0.0154 - val_loss: 0.0717 - val_mean_absolute_error: 0.0717
Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0144 - mean_absolute_error: 0.0144
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0144 - mean_absolute_error: 0.0144 - val_loss: 0.0622 - val_mean_absolute_error: 0.0622
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0097 - mean_absolute_error: 0.0097
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 57ms/step - loss: 0.0097 - mean_absolute_error: 0.0097 - val_loss: 0.0572 - val_mean_absolute_error: 0.0572
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0137 - mean_ab

In [20]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [21]:
y_pred=NN_model.predict(X_test)

1/1 [==============================] - 0s 225ms/step


In [22]:
r2_score(y_test,y_pred)

0.8958264213311867

In [23]:
y_test1 = sc_y.inverse_transform(y_test)
pred1 = sc_y.inverse_transform(y_pred.reshape(-1,1))
ann_prediction_test=get_preds(y_test1, pred1)
ann_prediction_test

,Actual,Predicted
0,7.52,7.748738
1,4.47,4.641231
2,5.81,6.284691
3,5.86,6.284691


In [24]:
ann_prediction_test["Actual"].corr(ann_prediction_test["Predicted"])

0.9932786871205617

In [26]:
from math import sqrt
print("TEST Mean absolute error :", mean_absolute_error(y_test1,pred1))
print("TEST Mean squared error :",mean_squared_error(y_test1,pred1))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(y_test1,pred1)))

TEST Mean absolute error : 0.3248376464843752
TEST Mean squared error : 0.12183369904888705
TEST Root Mean squared error : 0.3490468436311766


In [28]:
y_pred_train=NN_model.predict(X_train)

1/1 [==============================] - 0s 44ms/step


In [29]:
r2_score(y_train,y_pred_train)

0.9991404008567151

In [31]:
y_train1 = sc_y.inverse_transform(y_train)
pred_train= sc_y.inverse_transform(y_pred_train.reshape(-1,1))
ann_prediction_train=get_preds(y_train1, pred_train)
ann_prediction_train

,Actual,Predicted
0,6.93,6.937185
1,6.65,6.669899
2,4.58,4.574253
3,4.52,4.514845
4,6.46,6.454919
5,7.76,7.777654
6,7.22,7.236865
7,6.28,6.284691
8,6.45,6.451415
9,5.67,5.678446


In [32]:
ann_prediction_train["Actual"].corr(ann_prediction_train["Predicted"])

0.999744095567631

In [34]:
from math import sqrt
print("Train Mean absolute error :", mean_absolute_error(y_train1,pred_train))
print("Train Mean squared error :",mean_squared_error(y_train1,pred_train))
print("Train Root Mean squared error :",sqrt(mean_squared_error(y_train1,pred_train)))

Train Mean absolute error : 0.020278847034160864
Train Mean squared error : 0.0009245258316209896
Train Root Mean squared error : 0.030406016372109476


In [35]:
y_pred_real=NN_model.predict(X1)

1/1 [==============================] - 0s 57ms/step


In [36]:
sc_y.inverse_transform(y_pred_real)

array([[7.2368646],
       [7.748738 ],
       [4.574253 ],
       [6.451415 ],
       [4.641231 ],
       [5.678446 ],
       [6.454919 ],
       [7.777654 ],
       [6.669899 ],
       [4.5148454],
       [8.342107 ],
       [6.937185 ],
       [6.284691 ],
       [6.284691 ],
       [6.284691 ],
       [6.284691 ],
       [6.284691 ]], dtype=float32)

In [37]:
r2_score(y1,y_pred_real)

0.9742512401342994

In [38]:
y_real = sc_y.inverse_transform(y1)
pred_real= sc_y.inverse_transform(y_pred_real.reshape(-1,1))
ann_prediction_real=get_preds(y_real, pred_real)
ann_prediction_real

,Actual,Predicted
0,7.22,7.236865
1,7.52,7.748738
2,4.58,4.574253
3,6.45,6.451415
4,4.47,4.641231
5,5.67,5.678446
6,6.46,6.454919
7,7.76,7.777654
8,6.65,6.669899
9,4.52,4.514845


In [39]:
NN_model1= Sequential()

In [40]:
# The Input Layer :
NN_model1.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='leaky_relu'))

# The Hidden Layers :
NN_model1.add(Dense(64, kernel_initializer='normal',activation='leaky_relu'))
NN_model1.add(Dense(32, kernel_initializer='normal',activation='leaky_relu'))
NN_model1.add(Dense(16, kernel_initializer='normal',activation='leaky_relu'))

# The Output Layer :
NN_model1.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               512       
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [41]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint1 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list1 = [checkpoint1]

In [42]:
import time
start_time=time.time()
NN_model1.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list1)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8342 - mean_absolute_error: 0.8342
Epoch 1: val_loss improved from inf to 0.18419, saving model to Weights-001--0.18419.hdf5
1/1 [==============================] - 1s 1s/step - loss: 0.8342 - mean_absolute_error: 0.8342 - val_loss: 0.1842 - val_mean_absolute_error: 0.1842
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8337 - mean_absolute_error: 0.8337
Epoch 2: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 75ms/step - loss: 0.8337 - mean_absolute_error: 0.8337 - val_loss: 0.1856 - val_mean_absolute_error: 0.1856
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8332 - mean_absolute_error: 0.8332
Epoch 3: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 41ms/step - loss: 0.8332 - mean_absolute_error: 0.8332 - val_loss: 0.1870 - val_mean_absolute_error: 0.1870
Epoch 4/500
1/1 [==============================] - ETA: 

Epoch 28/500
1/1 [==============================] - ETA: 0s - loss: 0.7749 - mean_absolute_error: 0.7749
Epoch 28: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 51ms/step - loss: 0.7749 - mean_absolute_error: 0.7749 - val_loss: 0.2611 - val_mean_absolute_error: 0.2611
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.7674 - mean_absolute_error: 0.7674
Epoch 29: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 46ms/step - loss: 0.7674 - mean_absolute_error: 0.7674 - val_loss: 0.2677 - val_mean_absolute_error: 0.2677
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.7588 - mean_absolute_error: 0.7588
Epoch 30: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 46ms/step - loss: 0.7588 - mean_absolute_error: 0.7588 - val_loss: 0.2759 - val_mean_absolute_error: 0.2759
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.7494 - mean_absolute_

1/1 [==============================] - ETA: 0s - loss: 0.3433 - mean_absolute_error: 0.3433
Epoch 55: val_loss did not improve from 0.18419
1/1 [==============================] - 0s 45ms/step - loss: 0.3433 - mean_absolute_error: 0.3433 - val_loss: 0.1863 - val_mean_absolute_error: 0.1863
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.3198 - mean_absolute_error: 0.3198
Epoch 56: val_loss improved from 0.18419 to 0.16020, saving model to Weights-056--0.16020.hdf5
1/1 [==============================] - 0s 69ms/step - loss: 0.3198 - mean_absolute_error: 0.3198 - val_loss: 0.1602 - val_mean_absolute_error: 0.1602
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.2936 - mean_absolute_error: 0.2936
Epoch 57: val_loss improved from 0.16020 to 0.13567, saving model to Weights-057--0.13567.hdf5
1/1 [==============================] - 0s 68ms/step - loss: 0.2936 - mean_absolute_error: 0.2936 - val_loss: 0.1357 - val_mean_absolute_error: 0.1357
Epoch 58/5

Epoch 81/500
1/1 [==============================] - ETA: 0s - loss: 0.0688 - mean_absolute_error: 0.0688
Epoch 81: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 56ms/step - loss: 0.0688 - mean_absolute_error: 0.0688 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 82/500
1/1 [==============================] - ETA: 0s - loss: 0.0688 - mean_absolute_error: 0.0688
Epoch 82: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0688 - mean_absolute_error: 0.0688 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.0658 - mean_absolute_error: 0.0658
Epoch 83: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 51ms/step - loss: 0.0658 - mean_absolute_error: 0.0658 - val_loss: 0.0108 - val_mean_absolute_error: 0.0108
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.0545 - mean_absolute_

Epoch 108/500
1/1 [==============================] - ETA: 0s - loss: 0.0211 - mean_absolute_error: 0.0211
Epoch 108: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0211 - mean_absolute_error: 0.0211 - val_loss: 0.0774 - val_mean_absolute_error: 0.0774
Epoch 109/500
1/1 [==============================] - ETA: 0s - loss: 0.0157 - mean_absolute_error: 0.0157
Epoch 109: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0157 - mean_absolute_error: 0.0157 - val_loss: 0.0717 - val_mean_absolute_error: 0.0717
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0124 - mean_absolute_error: 0.0124
Epoch 110: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0124 - mean_absolute_error: 0.0124 - val_loss: 0.0676 - val_mean_absolute_error: 0.0676
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0193 - mean_ab

...

Epoch 378/500
1/1 [==============================] - ETA: 0s - loss: 0.0063 - mean_absolute_error: 0.0063
Epoch 378: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0063 - mean_absolute_error: 0.0063 - val_loss: 0.0671 - val_mean_absolute_error: 0.0671
Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0042 - mean_absolute_error: 0.0042
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 51ms/step - loss: 0.0042 - mean_absolute_error: 0.0042 - val_loss: 0.0727 - val_mean_absolute_error: 0.0727
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0121 - mean_absolute_error: 0.0121
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0121 - mean_absolute_error: 0.0121 - val_loss: 0.0716 - val_mean_absolute_error: 0.0716
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0091 - mean_ab

Epoch 405/500
1/1 [==============================] - ETA: 0s - loss: 0.0064 - mean_absolute_error: 0.0064
Epoch 405: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0064 - mean_absolute_error: 0.0064 - val_loss: 0.0693 - val_mean_absolute_error: 0.0693
Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.0078
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 47ms/step - loss: 0.0078 - mean_absolute_error: 0.0078 - val_loss: 0.0677 - val_mean_absolute_error: 0.0677
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0045 - mean_absolute_error: 0.0045
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 46ms/step - loss: 0.0045 - mean_absolute_error: 0.0045 - val_loss: 0.0655 - val_mean_absolute_error: 0.0655
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0047 - mean_ab

Epoch 432/500
1/1 [==============================] - ETA: 0s - loss: 0.0069 - mean_absolute_error: 0.0069
Epoch 432: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0069 - mean_absolute_error: 0.0069 - val_loss: 0.0634 - val_mean_absolute_error: 0.0634
Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0063 - mean_absolute_error: 0.0063
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 58ms/step - loss: 0.0063 - mean_absolute_error: 0.0063 - val_loss: 0.0615 - val_mean_absolute_error: 0.0615
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0043 - mean_absolute_error: 0.0043
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0043 - mean_absolute_error: 0.0043 - val_loss: 0.0642 - val_mean_absolute_error: 0.0642
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0056 - mean_ab

Epoch 459/500
1/1 [==============================] - ETA: 0s - loss: 0.0095 - mean_absolute_error: 0.0095
Epoch 459: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 57ms/step - loss: 0.0095 - mean_absolute_error: 0.0095 - val_loss: 0.0605 - val_mean_absolute_error: 0.0605
Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0074 - mean_absolute_error: 0.0074
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0074 - mean_absolute_error: 0.0074 - val_loss: 0.0660 - val_mean_absolute_error: 0.0660
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0046 - mean_absolute_error: 0.0046
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0046 - mean_absolute_error: 0.0046 - val_loss: 0.0685 - val_mean_absolute_error: 0.0685
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0085 - mean_ab

Epoch 486/500
1/1 [==============================] - ETA: 0s - loss: 0.0099 - mean_absolute_error: 0.0099
Epoch 486: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 46ms/step - loss: 0.0099 - mean_absolute_error: 0.0099 - val_loss: 0.0661 - val_mean_absolute_error: 0.0661
Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.0078
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 46ms/step - loss: 0.0078 - mean_absolute_error: 0.0078 - val_loss: 0.0601 - val_mean_absolute_error: 0.0601
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0040 - mean_absolute_error: 0.0040
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 73ms/step - loss: 0.0040 - mean_absolute_error: 0.0040 - val_loss: 0.0547 - val_mean_absolute_error: 0.0547
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0090 - mean_ab

In [44]:
NN_model1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [46]:
y_pred1=NN_model1.predict(X_test)
r2_score(y_test,y_pred1)

1/1 [==============================] - 0s 25ms/step


0.8859219324360962

In [47]:
y_pred_train1=NN_model1.predict(X_train)
r2_score(y_train,y_pred_train1)

1/1 [==============================] - 0s 31ms/step


0.9993149382611993

In [48]:
from math import sqrt
print("Train leaky Mean absolute error :", mean_absolute_error(y_train,y_pred_train1))
print("Train leaky Mean squared error :",mean_squared_error(y_train,y_pred_train1))
print("Train leaky Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train1)))

Train leaky Mean absolute error : 0.01584436525491447
Train leaky Mean squared error : 0.0006850617388006654
Train leaky Root Mean squared error : 0.026173684089189


In [49]:
y_pred_real1=NN_model1.predict(X1)
r2_score(y1,y_pred_real1)

1/1 [==============================] - 0s 33ms/step


0.9719878897560515

In [50]:
NN_model2= Sequential()

In [51]:
# The Input Layer :
NN_model2.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='exponential'))

# The Hidden Layers :
NN_model2.add(Dense(64, kernel_initializer='normal',activation='exponential'))
NN_model2.add(Dense(32, kernel_initializer='normal',activation='exponential'))
NN_model2.add(Dense(16, kernel_initializer='normal',activation='exponential'))

# The Output Layer :
NN_model2.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               512       
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 16)                528       
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [52]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint2 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list2 = [checkpoint2]

In [53]:
import time
start_time=time.time()
NN_model2.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list2)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8559 - mean_absolute_error: 0.8559
Epoch 1: val_loss improved from inf to 0.19710, saving model to Weights-001--0.19710.hdf5
1/1 [==============================] - 1s 1s/step - loss: 0.8559 - mean_absolute_error: 0.8559 - val_loss: 0.1971 - val_mean_absolute_error: 0.1971
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8311 - mean_absolute_error: 0.8311
Epoch 2: val_loss did not improve from 0.19710
1/1 [==============================] - 0s 75ms/step - loss: 0.8311 - mean_absolute_error: 0.8311 - val_loss: 0.2905 - val_mean_absolute_error: 0.2905
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8274 - mean_absolute_error: 0.8274
Epoch 3: val_loss did not improve from 0.19710
1/1 [==============================] - 0s 48ms/step - loss: 0.8274 - mean_absolute_error: 0.8274 - val_loss: 0.3393 - val_mean_absolute_error: 0.3393
Epoch 4/500
1/1 [==============================] - ETA: 

Epoch 28/500
1/1 [==============================] - ETA: 0s - loss: 1.2187 - mean_absolute_error: 1.2187
Epoch 28: val_loss did not improve from 0.19710
1/1 [==============================] - 0s 63ms/step - loss: 1.2187 - mean_absolute_error: 1.2187 - val_loss: 0.4408 - val_mean_absolute_error: 0.4408
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.6748 - mean_absolute_error: 0.6748
Epoch 29: val_loss did not improve from 0.19710
1/1 [==============================] - 0s 60ms/step - loss: 0.6748 - mean_absolute_error: 0.6748 - val_loss: 0.3222 - val_mean_absolute_error: 0.3222
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.7521 - mean_absolute_error: 0.7521
Epoch 30: val_loss did not improve from 0.19710
1/1 [==============================] - 0s 64ms/step - loss: 0.7521 - mean_absolute_error: 0.7521 - val_loss: 0.2529 - val_mean_absolute_error: 0.2529
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.7752 - mean_absolute_

1/1 [==============================] - 0s 50ms/step - loss: 0.7488 - mean_absolute_error: 0.7488 - val_loss: 0.2279 - val_mean_absolute_error: 0.2279
Epoch 55/500
1/1 [==============================] - ETA: 0s - loss: 0.7504 - mean_absolute_error: 0.7504
Epoch 55: val_loss did not improve from 0.14017
1/1 [==============================] - 0s 41ms/step - loss: 0.7504 - mean_absolute_error: 0.7504 - val_loss: 0.2205 - val_mean_absolute_error: 0.2205
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.7449 - mean_absolute_error: 0.7449
Epoch 56: val_loss did not improve from 0.14017
1/1 [==============================] - 0s 45ms/step - loss: 0.7449 - mean_absolute_error: 0.7449 - val_loss: 0.2282 - val_mean_absolute_error: 0.2282
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.7298 - mean_absolute_error: 0.7298
Epoch 57: val_loss did not improve from 0.14017
1/1 [==============================] - 0s 54ms/step - loss: 0.7298 - mean_absolute_error: 0

1/1 [==============================] - ETA: 0s - loss: 0.6083 - mean_absolute_error: 0.6083
Epoch 81: val_loss did not improve from 0.05453
1/1 [==============================] - 0s 43ms/step - loss: 0.6083 - mean_absolute_error: 0.6083 - val_loss: 0.1970 - val_mean_absolute_error: 0.1970
Epoch 82/500
1/1 [==============================] - ETA: 0s - loss: 0.6149 - mean_absolute_error: 0.6149
Epoch 82: val_loss did not improve from 0.05453
1/1 [==============================] - 0s 46ms/step - loss: 0.6149 - mean_absolute_error: 0.6149 - val_loss: 0.1917 - val_mean_absolute_error: 0.1917
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.6001 - mean_absolute_error: 0.6001
Epoch 83: val_loss did not improve from 0.05453
1/1 [==============================] - 0s 50ms/step - loss: 0.6001 - mean_absolute_error: 0.6001 - val_loss: 0.2104 - val_mean_absolute_error: 0.2104
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.5597 - mean_absolute_error: 0.5597

Epoch 108/500
1/1 [==============================] - ETA: 0s - loss: 0.3114 - mean_absolute_error: 0.3114
Epoch 108: val_loss did not improve from 0.02844
1/1 [==============================] - 0s 41ms/step - loss: 0.3114 - mean_absolute_error: 0.3114 - val_loss: 0.1382 - val_mean_absolute_error: 0.1382
Epoch 109/500
1/1 [==============================] - ETA: 0s - loss: 0.2864 - mean_absolute_error: 0.2864
Epoch 109: val_loss did not improve from 0.02844
1/1 [==============================] - 0s 43ms/step - loss: 0.2864 - mean_absolute_error: 0.2864 - val_loss: 0.2045 - val_mean_absolute_error: 0.2045
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.2979 - mean_absolute_error: 0.2979
Epoch 110: val_loss did not improve from 0.02844
1/1 [==============================] - 0s 43ms/step - loss: 0.2979 - mean_absolute_error: 0.2979 - val_loss: 0.2574 - val_mean_absolute_error: 0.2574
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.2878 - mean_ab

...

Epoch 378/500
1/1 [==============================] - ETA: 0s - loss: 0.1658 - mean_absolute_error: 0.1658
Epoch 378: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 54ms/step - loss: 0.1658 - mean_absolute_error: 0.1658 - val_loss: 0.4049 - val_mean_absolute_error: 0.4049
Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.2702 - mean_absolute_error: 0.2702
Epoch 379: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 53ms/step - loss: 0.2702 - mean_absolute_error: 0.2702 - val_loss: 0.2835 - val_mean_absolute_error: 0.2835
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.1471 - mean_absolute_error: 0.1471
Epoch 380: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 64ms/step - loss: 0.1471 - mean_absolute_error: 0.1471 - val_loss: 0.1560 - val_mean_absolute_error: 0.1560
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.2274 - mean_ab

Epoch 405/500
1/1 [==============================] - ETA: 0s - loss: 0.2198 - mean_absolute_error: 0.2198
Epoch 405: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 42ms/step - loss: 0.2198 - mean_absolute_error: 0.2198 - val_loss: 0.1249 - val_mean_absolute_error: 0.1249
Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.1942 - mean_absolute_error: 0.1942
Epoch 406: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 47ms/step - loss: 0.1942 - mean_absolute_error: 0.1942 - val_loss: 0.2331 - val_mean_absolute_error: 0.2331
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.1835 - mean_absolute_error: 0.1835
Epoch 407: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 39ms/step - loss: 0.1835 - mean_absolute_error: 0.1835 - val_loss: 0.2904 - val_mean_absolute_error: 0.2904
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.2035 - mean_ab

Epoch 432/500
1/1 [==============================] - ETA: 0s - loss: 0.1557 - mean_absolute_error: 0.1557
Epoch 432: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 48ms/step - loss: 0.1557 - mean_absolute_error: 0.1557 - val_loss: 0.2326 - val_mean_absolute_error: 0.2326
Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.1480 - mean_absolute_error: 0.1480
Epoch 433: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 52ms/step - loss: 0.1480 - mean_absolute_error: 0.1480 - val_loss: 0.1702 - val_mean_absolute_error: 0.1702
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.1840 - mean_absolute_error: 0.1840
Epoch 434: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 51ms/step - loss: 0.1840 - mean_absolute_error: 0.1840 - val_loss: 0.1357 - val_mean_absolute_error: 0.1357
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.1821 - mean_ab

Epoch 459/500
1/1 [==============================] - ETA: 0s - loss: 0.1571 - mean_absolute_error: 0.1571
Epoch 459: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 51ms/step - loss: 0.1571 - mean_absolute_error: 0.1571 - val_loss: 0.1088 - val_mean_absolute_error: 0.1088
Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.1159 - mean_absolute_error: 0.1159
Epoch 460: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 54ms/step - loss: 0.1159 - mean_absolute_error: 0.1159 - val_loss: 0.2324 - val_mean_absolute_error: 0.2324
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.1936 - mean_absolute_error: 0.1936
Epoch 461: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 48ms/step - loss: 0.1936 - mean_absolute_error: 0.1936 - val_loss: 0.1803 - val_mean_absolute_error: 0.1803
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.1733 - mean_ab

Epoch 486/500
1/1 [==============================] - ETA: 0s - loss: 0.1052 - mean_absolute_error: 0.1052
Epoch 486: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 57ms/step - loss: 0.1052 - mean_absolute_error: 0.1052 - val_loss: 0.1074 - val_mean_absolute_error: 0.1074
Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.1171 - mean_absolute_error: 0.1171
Epoch 487: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 53ms/step - loss: 0.1171 - mean_absolute_error: 0.1171 - val_loss: 0.1324 - val_mean_absolute_error: 0.1324
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.1158 - mean_absolute_error: 0.1158
Epoch 488: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 48ms/step - loss: 0.1158 - mean_absolute_error: 0.1158 - val_loss: 0.1229 - val_mean_absolute_error: 0.1229
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.1050 - mean_ab

In [54]:
NN_model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [55]:
y_pred2=NN_model2.predict(X_test)
r2_score(y_test,y_pred2)

1/1 [==============================] - 0s 121ms/step


0.8984887977034902

In [56]:
y_pred_train2=NN_model2.predict(X_train)
r2_score(y_train,y_pred_train2)

1/1 [==============================] - 0s 41ms/step


0.9696619979289265

In [57]:
from math import sqrt
print("Train exp Mean absolute error :", mean_absolute_error(y_train,y_pred_train2))
print("Train exp Mean squared error :",mean_squared_error(y_train,y_pred_train2))
print("Train exp Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train2)))

Train exp Mean absolute error : 0.13979492066706714
Train exp Mean squared error : 0.030338002071073535
Train exp Root Mean squared error : 0.17417807574741873


In [58]:
y_pred_real=NN_model2.predict(X1)

1/1 [==============================] - 0s 30ms/step


In [59]:
y_real1 = sc_y.inverse_transform(y1)
pred_real1= sc_y.inverse_transform(y_pred_real.reshape(-1,1))
ann_prediction_real1=get_preds(y_real1, pred_real1)
ann_prediction_real1

,Actual,Predicted
0,7.22,7.042217
1,7.52,7.430225
2,4.58,4.811107
3,6.45,6.421751
4,4.47,4.309617
5,5.67,5.994767
6,6.46,6.477343
7,7.76,7.762616
8,6.65,6.479105
9,4.52,4.703028


In [61]:
r2_score(y1,y_pred_real)

0.9536405090966025

In [62]:
NN_model3= Sequential()

In [63]:
# The Input Layer :
NN_model3.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='selu'))

# The Hidden Layers :
NN_model3.add(Dense(64, kernel_initializer='normal',activation='selu'))
NN_model3.add(Dense(32, kernel_initializer='normal',activation='selu'))
NN_model3.add(Dense(16, kernel_initializer='normal',activation='selu'))

# The Output Layer :
NN_model3.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model3.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               512       
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 32)                2080      
                                                                 
 dense_18 (Dense)            (None, 16)                528       
                                                                 
 dense_19 (Dense)            (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [64]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint3 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list3 = [checkpoint3]

In [65]:
NN_model3.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list2)

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8350 - mean_absolute_error: 0.8350
Epoch 1: val_loss did not improve from 0.01038
1/1 [==============================] - 2s 2s/step - loss: 0.8350 - mean_absolute_error: 0.8350 - val_loss: 0.1853 - val_mean_absolute_error: 0.1853
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8303 - mean_absolute_error: 0.8303
Epoch 2: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 77ms/step - loss: 0.8303 - mean_absolute_error: 0.8303 - val_loss: 0.1885 - val_mean_absolute_error: 0.1885
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8257 - mean_absolute_error: 0.8257
Epoch 3: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 74ms/step - loss: 0.8257 - mean_absolute_error: 0.8257 - val_loss: 0.1919 - val_mean_absolute_error: 0.1919
Epoch 4/500
1/1 [==============================] - ETA: 0s - loss: 0.8209 - mean_absolute_error: 0.

1/1 [==============================] - ETA: 0s - loss: 0.3040 - mean_absolute_error: 0.3040
Epoch 28: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 51ms/step - loss: 0.3040 - mean_absolute_error: 0.3040 - val_loss: 0.4407 - val_mean_absolute_error: 0.4407
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.2741 - mean_absolute_error: 0.2741
Epoch 29: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 50ms/step - loss: 0.2741 - mean_absolute_error: 0.2741 - val_loss: 0.4391 - val_mean_absolute_error: 0.4391
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.2472 - mean_absolute_error: 0.2472
Epoch 30: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 50ms/step - loss: 0.2472 - mean_absolute_error: 0.2472 - val_loss: 0.4280 - val_mean_absolute_error: 0.4280
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.2422 - mean_absolute_error: 0.2422

1/1 [==============================] - ETA: 0s - loss: 0.1206 - mean_absolute_error: 0.1206
Epoch 55: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 51ms/step - loss: 0.1206 - mean_absolute_error: 0.1206 - val_loss: 0.1471 - val_mean_absolute_error: 0.1471
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 56: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 56ms/step - loss: 0.1189 - mean_absolute_error: 0.1189 - val_loss: 0.1338 - val_mean_absolute_error: 0.1338
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.1165 - mean_absolute_error: 0.1165
Epoch 57: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 52ms/step - loss: 0.1165 - mean_absolute_error: 0.1165 - val_loss: 0.1252 - val_mean_absolute_error: 0.1252
Epoch 58/500
1/1 [==============================] - ETA: 0s - loss: 0.1120 - mean_absolute_error: 0.1120

1/1 [==============================] - ETA: 0s - loss: 0.0621 - mean_absolute_error: 0.0621
Epoch 82: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 77ms/step - loss: 0.0621 - mean_absolute_error: 0.0621 - val_loss: 0.0561 - val_mean_absolute_error: 0.0561
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.0582 - mean_absolute_error: 0.0582
Epoch 83: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 42ms/step - loss: 0.0582 - mean_absolute_error: 0.0582 - val_loss: 0.0515 - val_mean_absolute_error: 0.0515
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.0589 - mean_absolute_error: 0.0589
Epoch 84: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 46ms/step - loss: 0.0589 - mean_absolute_error: 0.0589 - val_loss: 0.0547 - val_mean_absolute_error: 0.0547
Epoch 85/500
1/1 [==============================] - ETA: 0s - loss: 0.0617 - mean_absolute_error: 0.0617

1/1 [==============================] - ETA: 0s - loss: 0.0352 - mean_absolute_error: 0.0352
Epoch 109: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 49ms/step - loss: 0.0352 - mean_absolute_error: 0.0352 - val_loss: 0.0496 - val_mean_absolute_error: 0.0496
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0433 - mean_absolute_error: 0.0433
Epoch 110: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 53ms/step - loss: 0.0433 - mean_absolute_error: 0.0433 - val_loss: 0.0600 - val_mean_absolute_error: 0.0600
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0375 - mean_absolute_error: 0.0375
Epoch 111: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 52ms/step - loss: 0.0375 - mean_absolute_error: 0.0375 - val_loss: 0.0809 - val_mean_absolute_error: 0.0809
Epoch 112/500
1/1 [==============================] - ETA: 0s - loss: 0.0394 - mean_absolute_error: 

...

Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0121 - mean_absolute_error: 0.0121
Epoch 379: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 53ms/step - loss: 0.0121 - mean_absolute_error: 0.0121 - val_loss: 0.0559 - val_mean_absolute_error: 0.0559
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0081 - mean_absolute_error: 0.0081
Epoch 380: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 54ms/step - loss: 0.0081 - mean_absolute_error: 0.0081 - val_loss: 0.0638 - val_mean_absolute_error: 0.0638
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_absolute_error: 0.0087
Epoch 381: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 60ms/step - loss: 0.0087 - mean_absolute_error: 0.0087 - val_loss: 0.0672 - val_mean_absolute_error: 0.0672
Epoch 382/500
1/1 [==============================] - ETA: 0s - loss: 0.0097 - mean_ab

Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0068 - mean_absolute_error: 0.0068
Epoch 406: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 61ms/step - loss: 0.0068 - mean_absolute_error: 0.0068 - val_loss: 0.0645 - val_mean_absolute_error: 0.0645
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0067 - mean_absolute_error: 0.0067
Epoch 407: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 57ms/step - loss: 0.0067 - mean_absolute_error: 0.0067 - val_loss: 0.0567 - val_mean_absolute_error: 0.0567
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0086 - mean_absolute_error: 0.0086
Epoch 408: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 58ms/step - loss: 0.0086 - mean_absolute_error: 0.0086 - val_loss: 0.0539 - val_mean_absolute_error: 0.0539
Epoch 409/500
1/1 [==============================] - ETA: 0s - loss: 0.0053 - mean_ab

Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0141 - mean_absolute_error: 0.0141
Epoch 433: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 70ms/step - loss: 0.0141 - mean_absolute_error: 0.0141 - val_loss: 0.0387 - val_mean_absolute_error: 0.0387
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0172 - mean_absolute_error: 0.0172
Epoch 434: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 66ms/step - loss: 0.0172 - mean_absolute_error: 0.0172 - val_loss: 0.0474 - val_mean_absolute_error: 0.0474
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0103 - mean_absolute_error: 0.0103
Epoch 435: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 74ms/step - loss: 0.0103 - mean_absolute_error: 0.0103 - val_loss: 0.0670 - val_mean_absolute_error: 0.0670
Epoch 436/500
1/1 [==============================] - ETA: 0s - loss: 0.0089 - mean_ab

Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0067 - mean_absolute_error: 0.0067
Epoch 460: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 44ms/step - loss: 0.0067 - mean_absolute_error: 0.0067 - val_loss: 0.0606 - val_mean_absolute_error: 0.0606
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0137 - mean_absolute_error: 0.0137
Epoch 461: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 51ms/step - loss: 0.0137 - mean_absolute_error: 0.0137 - val_loss: 0.0606 - val_mean_absolute_error: 0.0606
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0089 - mean_absolute_error: 0.0089
Epoch 462: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 55ms/step - loss: 0.0089 - mean_absolute_error: 0.0089 - val_loss: 0.0615 - val_mean_absolute_error: 0.0615
Epoch 463/500
1/1 [==============================] - ETA: 0s - loss: 0.0107 - mean_ab

Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0062 - mean_absolute_error: 0.0062
Epoch 487: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 45ms/step - loss: 0.0062 - mean_absolute_error: 0.0062 - val_loss: 0.0566 - val_mean_absolute_error: 0.0566
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0064 - mean_absolute_error: 0.0064
Epoch 488: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 46ms/step - loss: 0.0064 - mean_absolute_error: 0.0064 - val_loss: 0.0573 - val_mean_absolute_error: 0.0573
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0062 - mean_absolute_error: 0.0062
Epoch 489: val_loss did not improve from 0.01038
1/1 [==============================] - 0s 45ms/step - loss: 0.0062 - mean_absolute_error: 0.0062 - val_loss: 0.0631 - val_mean_absolute_error: 0.0631
Epoch 490/500
1/1 [==============================] - ETA: 0s - loss: 0.0046 - mean_ab

In [66]:
NN_model3.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [68]:
y_pred3=NN_model3.predict(X_test)
y_pred_train3=NN_model3.predict(X_train)
r2_score(y_train,y_pred_train3) , r2_score(y_test,y_pred3)

1/1 [==============================] - 0s 26ms/step


(0.9986683475136278, 0.8209549078836651)

In [69]:
print("Train selu Mean absolute error :", mean_absolute_error(y_train,y_pred_train3))
print("Train selu Mean squared error :",mean_squared_error(y_train,y_pred_train3))
print("Train selu Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train3)))

Train selu Mean absolute error : 0.028830906958121037
Train selu Mean squared error : 0.0013316524863722333
Train selu Root Mean squared error : 0.036491813963849935


In [70]:
y_pred_real3=NN_model3.predict(X1)
r2_score(y1,y_pred_real3)

1/1 [==============================] - 0s 39ms/step


0.9558502074289333